In [ ]:
#-*- coding:UTF-8 -*-
import requests,re, json, sys
from bs4 import BeautifulSoup
from urllib import request

class video_downloader():
    def __init__(self, url):
        self.server = 'http://jx.598110.com'
        self.api = 'http://jx.598110.com/?url='
        #self.get_url_api = 'http://api.xfsub.com/xfsub_api/url.php'
        self.url = url.split('#')[0]
        self.target = self.api + self.url
        self.s = requests.session()

    """
    函数说明:获取key、time、url等参数
    Parameters:
        无
    Returns:
        无
    Modify:
        2017-09-18
    """
    def get_key(self):
        req = self.s.get(url=self.target)
        req.encoding = 'utf-8'
        self.info = json.loads(re.findall('"url.php",\ (.+),', req.text)[0])    #使用正则表达式匹配结果，将匹配的结果存入info变量中

    """
    函数说明:获取视频地址
    Parameters:
        无
    Returns:
        video_url - 视频存放地址
    Modify:
        2017-09-18
    """
    def get_url(self):
        data = {'time':self.info['time'],
            'key':self.info['key'],
            'url':self.info['url'],
            'type':''}
        req = self.s.post(url=self.get_url_api,data=data)
        url = self.server + json.loads(req.text)['url']
        req = self.s.get(url)
        bf = BeautifulSoup(req.text,'xml')                                        #因为文件是xml格式的，所以要进行xml解析。
        video_url = bf.find('file').string                                        #匹配到视频地址
        return video_url

    """
    函数说明:回调函数，打印下载进度
    Parameters:
        a b c - 返回信息
    Returns:
        无
    Modify:
        2017-09-18
    """
    def Schedule(self, a, b, c):
        per = 100.0*a*b/c
        if per > 100 :
            per = 1
        sys.stdout.write("  " + "%.2f%% 已经下https://jx.618g.com/?url=https://www.iqiyi.com/v_19rr839kro.html?fc=b3604d2dd54c307f载的大小:%ld 文件大小:%ld" % (per,a*b,c) + '\r')
        sys.stdout.flush()

    """
    函数说明:视频下载
    Parameters:
        url - 视频地址
        filename - 视频名字
    Returns:
        无
    Modify:
        2017-09-18
    """
    def video_download(self, url, filename):
        request.urlretrieve(url=url,filename=filename,reporthook=self.Schedule)


if __name__ == '__main__':
    url = 'https://www.iqiyi.com/v_19rr839kro.html?fc=b3604d2dd54c307f#vfrm=19-9-0-1'
    vd = video_downloader(url)
    filename = '唐人街探案2'
    print('%s下载中:' % filename)
    vd.get_key()
    video_url = vd.get_url()
    print('  获取地址成功:%s' % video_url)
    vd.video_download(video_url, filename+'.mp4')
    print('\n下载完成！')

In [39]:

# -*- coding:utf-8 -*-  
import os
import sys
import requests
import urllib3
import datetime
from bs4 import BeautifulSoup

#from Crypto.Cipher import AES
from binascii import b2a_hex, a2b_hex
 
#reload(sys)
#sys.setdefaultencoding('utf-8')

def geturl(yurl):
    yurl = 'http://jx.618g.com/?url='+ yurl
    kv = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
    r = requests.get(yurl,headers = kv,verify = False).text
    r_bq = BeautifulSoup(r,'html.parser')
    sc = r_bq.find_all('div' ,id="a1" )[0]
    sc_bq = BeautifulSoup(str(sc),'html.parser')
    ifr = r_bq.find_all('iframe')[0]
    url0 = ifr.get('src').rsplit('=')[-1]
    url1 = requests.get(url0,headers = kv,verify = False).text
    url = url0.rsplit('/',1)[0]+'/'+url1.split("\n")[-1]
    
    
    
    print(url)
    return url
    
    
    
    
def download(url):
    

    
    kv = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
    download_path = os.getcwd() + "\download"  #os.getcwd()当前目录
    if not os.path.exists(download_path):
        os.mkdir(download_path)
        
    #新建日期文件夹
    download_path = os.path.join(download_path, datetime.datetime.now().strftime('%Y%m%d_%H%M%S'))
    #print download_path
    os.mkdir(download_path)
        
    all_content = requests.get(url,headers = kv,verify = False).text  # 获取第一层M3U8文件内容
    if "#EXTM3U" not in all_content:
        raise BaseException("非M3U8的链接")
 
    if "EXT-X-STREAM-INF" in all_content:  # 第一层
        file_line = all_content.split("\n")
        for line in file_line:
            if '.m3u8' in line:
                url = url.rsplit("/", 1)[0] + "/" + line # 拼出第二层m3u8的URL
                all_content = requests.get(url,headers = kv,verify = False).text
 
    file_line = all_content.split("\n")
 
    unknow = True
    key = ""
    for index, line in enumerate(file_line): # 第二层 enumerate()组合为一个索引序列
        if "#EXT-X-KEY" in line:  # 找解密Key
            method_pos = line.find("METHOD")
            comma_pos = line.find(",")
            method = line[method_pos:comma_pos].split('=')[1]
            print("Decode Method：", method)
            
            uri_pos = line.find("URI")
            quotation_mark_pos = line.rfind('"')
            key_path = line[uri_pos:quotation_mark_pos].split('"')[1]
            
            key_url = url.rsplit("/", 1)[0] + "/" + key_path # 拼出key解密密钥URL
            res = requests.get(key_url,verify = False)
            key = res.content
            print("key：" , key) 
            
        if "EXTINF" in line: # 找ts地址并下载
            unknow = False
            pd_url = url.rsplit("/", 1)[0] + "/" + file_line[index + 1] # 拼出ts片段的URL
            #print pd_url
            
            res = requests.get(pd_url,headers = kv,verify = False)
            c_fule_name = file_line[index + 1].rsplit("/", 1)[-1]
            
            if len(key): # AES 解密
                cryptor = AES.new(key, AES.MODE_CBC, key)  
                with open(os.path.join(download_path, c_fule_name + ".mp4"), 'ab') as f:
                    f.write(cryptor.decrypt(res.content))
            else:
                with open(os.path.join(download_path, c_fule_name), 'ab') as f:
                    f.write(res.content)
                    f.flush()
            print('当前下载进度:{:.1f}%' .format(index / len(file_line) * 100))
    if unknow:
        raise BaseException("未找到对应的下载链接")
    else:
        print("下载完成") 
    merge_file(download_path)

def merge_file(path):
    os.chdir(path)  #改变当前工作目录到指定的路径。
    cmd = "copy /b * new.tmp"
    os.system(cmd)  #来执行系统命令
    os.system('del /Q *.ts')
    os.system('del /Q *.mp4')
    os.rename("new.tmp", "new.mp4")
    
if __name__ == '__main__': 
    
    #url = "http://youku.cdn-iqiyi.com/20180420/8807_20a46f3f/1000k/hls/index.m3u8"
    yurl = 'https://www.iqiyi.com/v_19rr7pkf7o.html'
    url = geturl(yurl)
    download(url)



https://sohu.com-v-sohu.com/20180920/12867_4d6b4f20/800k/hls/index.m3u8
当前下载进度:0.1%
当前下载进度:0.2%
当前下载进度:0.3%
当前下载进度:0.3%
当前下载进度:0.4%
当前下载进度:0.5%
当前下载进度:0.5%
当前下载进度:0.6%
当前下载进度:0.7%
当前下载进度:0.7%
当前下载进度:0.8%
当前下载进度:0.9%
当前下载进度:0.9%
当前下载进度:1.0%
当前下载进度:1.1%
当前下载进度:1.2%
当前下载进度:1.2%
当前下载进度:1.3%
当前下载进度:1.4%
当前下载进度:1.4%
当前下载进度:1.5%
当前下载进度:1.6%
当前下载进度:1.6%
当前下载进度:1.7%
当前下载进度:1.8%
当前下载进度:1.8%
当前下载进度:1.9%
当前下载进度:2.0%
当前下载进度:2.0%
当前下载进度:2.1%
当前下载进度:2.2%
当前下载进度:2.2%
当前下载进度:2.3%
当前下载进度:2.4%
当前下载进度:2.4%
当前下载进度:2.5%
当前下载进度:2.6%
当前下载进度:2.6%
当前下载进度:2.7%
当前下载进度:2.8%
当前下载进度:2.8%
当前下载进度:2.9%
当前下载进度:3.0%
当前下载进度:3.0%
当前下载进度:3.1%
当前下载进度:3.2%
当前下载进度:3.2%
当前下载进度:3.3%
当前下载进度:3.4%
当前下载进度:3.5%
当前下载进度:3.5%
当前下载进度:3.6%
当前下载进度:3.7%
当前下载进度:3.7%
当前下载进度:3.8%
当前下载进度:3.9%
当前下载进度:3.9%
当前下载进度:4.0%
当前下载进度:4.1%
当前下载进度:4.1%
当前下载进度:4.2%
当前下载进度:4.3%
当前下载进度:4.3%
当前下载进度:4.4%
当前下载进度:4.5%
当前下载进度:4.5%
当前下载进度:4.6%
当前下载进度:4.7%
当前下载进度:4.7%
当前下载进度:4.8%
当前下载进度:4.9%
当前下载进度:4.9%
当前下载进度:5.0%
当前下载进度:5.1%
当前下载进度:5.1%
当前下载进度:5.2%
当前下载进度:5.3%
当前下载

当前下载进度:43.2%
当前下载进度:43.2%
当前下载进度:43.3%
当前下载进度:43.4%
当前下载进度:43.4%
当前下载进度:43.5%
当前下载进度:43.6%
当前下载进度:43.6%
当前下载进度:43.7%
当前下载进度:43.8%
当前下载进度:43.8%
当前下载进度:43.9%
当前下载进度:44.0%
当前下载进度:44.0%
当前下载进度:44.1%
当前下载进度:44.2%
当前下载进度:44.2%
当前下载进度:44.3%
当前下载进度:44.4%
当前下载进度:44.5%
当前下载进度:44.5%
当前下载进度:44.6%
当前下载进度:44.7%
当前下载进度:44.7%
当前下载进度:44.8%
当前下载进度:44.9%
当前下载进度:44.9%
当前下载进度:45.0%
当前下载进度:45.1%
当前下载进度:45.1%
当前下载进度:45.2%
当前下载进度:45.3%
当前下载进度:45.3%
当前下载进度:45.4%
当前下载进度:45.5%
当前下载进度:45.5%
当前下载进度:45.6%
当前下载进度:45.7%
当前下载进度:45.7%
当前下载进度:45.8%
当前下载进度:45.9%
当前下载进度:45.9%
当前下载进度:46.0%
当前下载进度:46.1%
当前下载进度:46.1%
当前下载进度:46.2%
当前下载进度:46.3%
当前下载进度:46.3%
当前下载进度:46.4%
当前下载进度:46.5%
当前下载进度:46.5%
当前下载进度:46.6%
当前下载进度:46.7%
当前下载进度:46.8%
当前下载进度:46.8%
当前下载进度:46.9%
当前下载进度:47.0%
当前下载进度:47.0%
当前下载进度:47.1%
当前下载进度:47.2%
当前下载进度:47.2%
当前下载进度:47.3%
当前下载进度:47.4%
当前下载进度:47.4%
当前下载进度:47.5%
当前下载进度:47.6%
当前下载进度:47.6%
当前下载进度:47.7%
当前下载进度:47.8%
当前下载进度:47.8%
当前下载进度:47.9%
当前下载进度:48.0%
当前下载进度:48.0%
当前下载进度:48.1%
当前下载进度:48.2%
当前下载进度:48.2%
当前下载进度:48.3%

当前下载进度:85.9%
当前下载进度:85.9%
当前下载进度:86.0%
当前下载进度:86.1%
当前下载进度:86.1%
当前下载进度:86.2%
当前下载进度:86.3%
当前下载进度:86.3%
当前下载进度:86.4%
当前下载进度:86.5%
当前下载进度:86.5%
当前下载进度:86.6%
当前下载进度:86.7%
当前下载进度:86.7%
当前下载进度:86.8%
当前下载进度:86.9%
当前下载进度:86.9%
当前下载进度:87.0%
当前下载进度:87.1%
当前下载进度:87.1%
当前下载进度:87.2%
当前下载进度:87.3%
当前下载进度:87.3%
当前下载进度:87.4%
当前下载进度:87.5%
当前下载进度:87.6%
当前下载进度:87.6%
当前下载进度:87.7%
当前下载进度:87.8%
当前下载进度:87.8%
当前下载进度:87.9%
当前下载进度:88.0%
当前下载进度:88.0%
当前下载进度:88.1%
当前下载进度:88.2%
当前下载进度:88.2%
当前下载进度:88.3%
当前下载进度:88.4%
当前下载进度:88.4%
当前下载进度:88.5%
当前下载进度:88.6%
当前下载进度:88.6%
当前下载进度:88.7%
当前下载进度:88.8%
当前下载进度:88.8%
当前下载进度:88.9%
当前下载进度:89.0%
当前下载进度:89.0%
当前下载进度:89.1%
当前下载进度:89.2%
当前下载进度:89.2%
当前下载进度:89.3%
当前下载进度:89.4%
当前下载进度:89.4%
当前下载进度:89.5%
当前下载进度:89.6%
当前下载进度:89.6%
当前下载进度:89.7%
当前下载进度:89.8%
当前下载进度:89.9%
当前下载进度:89.9%
当前下载进度:90.0%
当前下载进度:90.1%
当前下载进度:90.1%
当前下载进度:90.2%
当前下载进度:90.3%
当前下载进度:90.3%
当前下载进度:90.4%
当前下载进度:90.5%
当前下载进度:90.5%
当前下载进度:90.6%
当前下载进度:90.7%
当前下载进度:90.7%
当前下载进度:90.8%
当前下载进度:90.9%
当前下载进度:90.9%
当前下载进度:91.0%

In [45]:

# -*- coding:utf-8 -*-  
import os
import sys
import requests
import urllib3
import datetime
from bs4 import BeautifulSoup

#from Crypto.Cipher import AES
from binascii import b2a_hex, a2b_hex
 
#reload(sys)
#sys.setdefaultencoding('utf-8')

def geturl(yurl):
    yurl = 'http://jx.618g.com/?url='+ yurl
    kv = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
    r = requests.get(yurl,headers = kv,verify = False).text
    r_bq = BeautifulSoup(r,'html.parser')
    sc = r_bq.find_all('div' ,id="a1" )[0]
    sc_bq = BeautifulSoup(str(sc),'html.parser')
    ifr = r_bq.find_all('iframe')[0]
    url0 = ifr.get('src').rsplit('=')[-1]
    url1 = requests.get(url0,headers = kv,verify = False).text
    url = url0.rsplit('/',1)[0]+'/'+url1.split("\n")[-1]
    
    
    
    print(url)
    return url
    
    
    
    
def download(url):
    

    
    kv = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
    download_path = 'E:\pythonxuexi\download' #os.getcwd()当前目录
    if not os.path.exists(download_path):
        os.mkdir(download_path)
        
    #新建日期文件夹
    download_path = os.path.join(download_path, datetime.datetime.now().strftime('%Y%m%d_%H%M%S'))
    #print download_path
    os.mkdir(download_path)
        
    all_content = requests.get(url,headers = kv,verify = False).text  # 获取第一层M3U8文件内容
    if "#EXTM3U" not in all_content:
        raise BaseException("非M3U8的链接")
 
    if "EXT-X-STREAM-INF" in all_content:  # 第一层
        file_line = all_content.split("\n")
        for line in file_line:
            if '.m3u8' in line:
                url = url.rsplit("/", 1)[0] + "/" + line # 拼出第二层m3u8的URL
                all_content = requests.get(url,headers = kv,verify = False).text
 
    file_line = all_content.split("\n")
 
    unknow = True
    key = ""
    for index, line in enumerate(file_line): # 第二层 enumerate()组合为一个索引序列
        if "#EXT-X-KEY" in line:  # 找解密Key
            method_pos = line.find("METHOD")
            comma_pos = line.find(",")
            method = line[method_pos:comma_pos].split('=')[1]
            print("Decode Method：", method)
            
            uri_pos = line.find("URI")
            quotation_mark_pos = line.rfind('"')
            key_path = line[uri_pos:quotation_mark_pos].split('"')[1]
            
            key_url = url.rsplit("/", 1)[0] + "/" + key_path # 拼出key解密密钥URL
            res = requests.get(key_url,verify = False)
            key = res.content
            print("key：" , key) 
            
        if "EXTINF" in line: # 找ts地址并下载
            unknow = False
            pd_url = url.rsplit("/", 1)[0] + "/" + file_line[index + 1] # 拼出ts片段的URL
            #print pd_url
            
            res = requests.get(pd_url,headers = kv,verify = False)
            c_fule_name = file_line[index + 1].rsplit("/", 1)[-1]
            
            if len(key): # AES 解密
                cryptor = AES.new(key, AES.MODE_CBC, key)  
                with open(os.path.join(download_path, c_fule_name + ".mp4"), 'ab') as f:
                    f.write(cryptor.decrypt(res.content))
            else:
                with open(os.path.join(download_path, c_fule_name), 'ab') as f:
                    f.write(res.content)
                    f.flush()
            print('当前下载进度:{:.1f}%' .format(index / len(file_line) * 100))
    if unknow:
        raise BaseException("未找到对应的下载链接")
    else:
        print("下载完成") 
    merge_file(download_path)

def merge_file(path):
    os.chdir(path)  #改变当前工作目录到指定的路径。
    cmd = "copy /b * new.tmp"
    os.system(cmd)  #来执行系统命令
    os.system('del /Q *.ts')
    os.system('del /Q *.mp4')
    os.rename("new.tmp", "new.mp4")
    
if __name__ == '__main__': 
    
    #url = "http://youku.cdn-iqiyi.com/20180420/8807_20a46f3f/1000k/hls/index.m3u8"
    yurl = 'https://www.bilibili.com/video/av32239788/?spm_id_from=333.334.chief_recommend.22'
    url = geturl(yurl)
    download(url)

https://www.bilibili.com/video/av32239788/Effect：そぼろikeno</div><!----></div><!----><div id="recommend_report" class="video-like-m y-layout report-wrap-module report-scroll-module"><p class="title">看过该视频的还喜欢</p><div class="like-list clearfix"><div class="load-state"><span class="loading">正在加载...</span><!----><!----></div><!----><!----></div></div><div id="comment" class="comment-m y-layout comment-layout report-scroll-module"><div class="common"><div class="b-head"><span class="b-head-t results"></span><span class="b-head-t">评论</span><a href="//www.bilibili.com/replydeletelog?aid=32239788&amp;title=【社保BAAM】大晚上别跳了❤回来睡觉了" target="_blank" class="del-log">查看删除日志</a></div><div class="comment"></div></div></div></div><div class="r-wrapper"><!----><!----><div id="fixnav_report" class="fixed-nav-m report-wrap-module report-scroll-module" style="display:none;"><div title="返回顶部" class="go-top icon"></div><div title="点击打开迷你播放器" class="mini">mini<br>OFF</div></div><!----></div></div><!----><!----><

BaseException: 非M3U8的链接